In [ ]:
import pandas as pd
import glob
import os
import numpy as np

# Preprocessing

- Delete Magnıtude and Phase subfolders
- FOR /d /r . %d IN (Magnitude) DO @IF EXIST "%d" rd /s /q "%d"
- Use Medical Image Converter to convert files

In [ ]:
def readLabelFiles(path):
    files = os.listdir(path)
    data={}
    for i in files:
        df=pd.read_excel(path+i,header=None)
        name=i.split('.')
        data[name[0]]=df
    return data

In [ ]:
data=readLabelFiles("C:/Users/PROTED/Downloads/CMBs Dataset/CMBs Dataset/HR Labels/")

In [ ]:
def getSliceNumbers(data):
    sliceN={}
    for k in data.keys():
        sliceN[k]=data[k].values[:,0]
    return sliceN


In [ ]:
sliceNumbers=getSliceNumbers(data)

In [ ]:
#!pip install --upgrade Pillow

In [ ]:
from PIL import Image
#img=Image.open("C:/Users/PROTED/Downloads/ConvertedNifti/P_001-0001.jpg")
#img.show()

In [ ]:
def importImages(path):
    allImages=[]
    files = os.listdir(path)
    for i in files:
        allImages.append(Image.open(path+i))
    df=pd.DataFrame({'Name':files,'Image':allImages})
    return df

In [ ]:
Images=importImages("C:/Users/PROTED/Downloads/ConvertedNifti/")

In [ ]:
Images.head()

,Name,Image
0,P_001-0001.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...
1,P_001-0002.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...
2,P_001-0003.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...
3,P_001-0004.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...
4,P_001-0005.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...


In [ ]:
#Add slice number to Images
sliceN=[]
for i in Images.Name:
    sliceN.append(i.split("_")[1].replace(".jpg",""))
Images["Slice"]=sliceN

In [ ]:
Images.head()

,Name,Image,Slice
0,P_001-0001.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,001-0001
1,P_001-0002.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,001-0002
2,P_001-0003.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,001-0003
3,P_001-0004.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,001-0004
4,P_001-0005.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,001-0005


In [ ]:
#Slices with cmbs
cmbSlices=[]
for k in sliceNumbers.keys():
    a=k.split("_")[1]
    for i in sliceNumbers[k]:
        if i>=10:
            cmbSlices.append(a+"-00"+str(i))
        else:
            cmbSlices.append(a+"-000"+str(i))



In [ ]:
cmbImages=[]
for i in cmbSlices:
    cmbImages.append(Images[Images['Slice']==i]['Image'].values)
cmbImages

[array([<PIL.JpegImagePlugin.JpegImageFile image mode=L size=448x512 at 0x21D229B4910>],
       dtype=object),
 array([<PIL.JpegImagePlugin.JpegImageFile image mode=L size=448x512 at 0x21D229B4970>],
       dtype=object),
 array([<PIL.JpegImagePlugin.JpegImageFile image mode=L size=448x512 at 0x21D229FA5B0>],
       dtype=object),
 array([<PIL.JpegImagePlugin.JpegImageFile image mode=L size=448x512 at 0x21D229FA610>],
       dtype=object),
 array([<PIL.JpegImagePlugin.JpegImageFile image mode=L size=448x512 at 0x21D22A1D9D0>],
       dtype=object),
 array([<PIL.JpegImagePlugin.JpegImageFile image mode=L size=448x512 at 0x21D22A1DA30>],
       dtype=object),
 array([<PIL.JpegImagePlugin.JpegImageFile image mode=L size=448x512 at 0x21D22A64A90>],
       dtype=object),
 array([<PIL.JpegImagePlugin.JpegImageFile image mode=L size=448x512 at 0x21D22A64AF0>],
       dtype=object),
 array([<PIL.JpegImagePlugin.JpegImageFile image mode=L size=448x512 at 0x21D22A64B50>],
       dtype=object),
 

In [ ]:
#Save the Images
for i in range(len(cmbImages)):
    #print(i)
    cmbImages[i][0].save("C:/Users/PROTED/Downloads/cmbImages/"+cmbSlices[i]+".jpg")


In [ ]:
 #I = np.asarray(Image.open("C:/Users/PROTED/Downloads/cmbImages/001-0049.jpg"))

In [ ]:
#Generate the text file
#Let's get x and y coordinates for each slice
def getX(data):
    X={}
    for k in data.keys():
        X[k]=data[k].values[:,1]
    return X
x=getX(data)
def getY(data):
    Y={}
    for k in data.keys():
        Y[k]=data[k].values[:,2]
    return Y
y=getY(data)

# YOLO Steps

0.Download pretraining weigths https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
1../cfg file is already avaiable.Put it in ./cfg

In [ ]:
#2. build\darknet\x64\data\
with open(os.path.join("C:/Users/PROTED/Downloads/data/obj.names"),"w") as f:
    f.write("CMB")

In [ ]:
#3.build\darknet\x64\data\
with open(os.path.join("C:/Users/PROTED/Downloads/data/obj.data"),"w") as f:
    f.write("classes = 1\n")
    f.write("train = data/train.txt\n")
    f.write("valid = data/test.txt\n")
    f.write("names = data/obj.names\n")
    f.write("backup/")

4. Put jpgs in build\darknet\x64\data\obj\

In [ ]:
#5.Writing text files
#labels same directory as images
height=448
width=512
X=np.concatenate(list(x.values()), axis=0 )
Y=np.concatenate(list(y.values()), axis=0 )
for s in range(len(cmbSlices)):
    with open(os.path.join("C:/Users/PROTED/Downloads/textFiles/","{}.txt".format(cmbSlices[s])),"w") as f:
        f.write("0 "+str((X[s]/512))+" "+str(float(Y[s]/448))+" "+str(float(20/512)) +" " +str(float(20/448))+"\n")

6. train and test files

In [ ]:
#Lets do an 80 to 20 split
#Create train.txt
with open(os.path.join("C:/Users/PROTED/Downloads/data/train.txt"),"w") as f:
        for i in cmbSlices[1:152]:
            f.write("data/obj/"+i+".jpg\n")

In [ ]:
#Create test.txt
with open(os.path.join("C:/Users/PROTED/Downloads/data/test.txt"),"w") as f:
        for i in cmbSlices[152:len(cmbSlices)]:
            f.write("data/obj/"+i+".jpg\n")

7. Download pre-trained weights for the convolutional layers and put to the directory build\darknet\x64. Same as step 0

8. Begin training
darknet.exe detector train data/obj.data yolo-obj.cfg yolov4.conv.137

9. After training is complete - get result yolo-obj_final.weights from path build\darknet\x64\backup\